# 树回归
CART(Classification And Regression Trees， 分类回归树)，该算法既可以用于分类还可以用于回归。

## 场景
我们在第 8 章中介绍了线性回归的一些强大的方法，但这些方法创建的模型需要拟合所有的样本点（局部加权线性回归除外）。当数据拥有众多特征并且特征之间关系十分复杂时，构建全局模型的想法就显得太难了，也略显笨拙。而且，实际生活中很多问题都是非线性的，不可能使用全局线性模型来拟合任何数据。

## 原理
### 概述
为了构建以分段常数为叶节点的树，我们需要度量数据的一致性。对于分类问题，可以通过计算每个节点的数据混乱度来衡量一致性。然而，当数据是连续型数值时，计算混乱度的方法有所不同。

在处理连续数据时，混乱度的计算方法如下：

1. **计算均值**：假设数据集为 $D = \{x_1, x_2, \dots, x_n\}$，数据集的均值可以表示为：

   $$
   \mu = \frac{1}{n} \sum_{i=1}^n x_i
   $$

2. **计算差值**：对于每个数据点 $x_i$，计算其与均值的差值 $x_i - \mu$。

3. **计算总方差**：为了度量数据的一致性，可以使用每个差值的平方和，这种方法类似于统计学中的方差计算。总方差可以表示为：

   $$
   \text{总方差} = \sum_{i=1}^n (x_i - \mu)^2
   $$

   总方差衡量了数据的总混乱度。与方差不同的是，方差是每个差值的平方均值，而我们关心的是这些平方差的总和。

4. **均方差与总方差的关系**：如果需要转换为均方差（即方差），可以将总方差除以数据集的样本数 $n$：

   $$
   \text{均方差} = \frac{1}{n} \sum_{i=1}^n (x_i - \mu)^2
   $$

   反之，已知均方差的情况下，总方差可以通过均方差乘以数据集的大小 $n$ 得到：

   $$
   \text{总方差} = n \times \text{均方差}
   $$

通过总方差，我们可以有效地衡量连续数据的混乱度，进而判断数据的一致性。这为构建以分段常数为叶节点的树提供了一个有效的标准。

### 不同树构建算法的比较
决策树构建算法在机器学习中非常重要，主要用于分类和回归任务。常见的决策树算法包括 **ID3**、**C4.5** 和 **CART**。这些算法的差异主要在于**特征选择方法**和**分支策略**。以下是对这些算法的详细介绍。

#### 1. ID3（Iterative Dichotomiser 3）

##### 信息熵 (Entropy)
对于一个数据集 $D$，假设有 $k$ 个类别，每个类别 $i$ 的样本占比为 $p_i$，则数据集的**信息熵** $H(D)$ 定义为：

$$
H(D) = -\sum_{i=1}^k p_i \log_2(p_i)
$$

##### 条件熵 (Conditional Entropy)
假设我们选定一个特征 $A$ 进行数据集划分，该特征有 $v$ 个可能的取值 $\{a_1, a_2, \dots, a_v\}$，其中 $D_j$ 表示特征 $A$ 取值为 $a_j$ 的样本子集。那么，特征 $A$ 的**条件熵** $H(D|A)$ 表示在已知特征 $A$ 的条件下数据集 $D$ 的熵，定义为：

$$
H(D|A) = \sum_{j=1}^v \frac{|D_j|}{|D|} H(D_j)
$$

##### 信息增益 (Information Gain)
特征 $A$ 的**信息增益** $IG(D, A)$ 定义为原数据集的熵减去在特征 $A$ 条件下的熵：

$$
IG(D, A) = H(D) - H(D|A)
$$

**ID3** 算法是一种基于**信息增益**的决策树算法，其主要步骤如下：

- **特征选择**：ID3 每次选择具有最高信息增益的特征来分割数据。信息增益用于衡量某个特征在分割数据后能带来的混乱度减少，信息增益越大，表示该特征在分类时越重要。
  
- **分支策略**：ID3 使用多叉分支，也就是说，如果一个特征有 4 个取值，则会生成 4 个分支。分支越多，树的深度可能越小，但由于分割较快，容易导致过拟合。
  
- **连续型特征处理**：ID3 无法直接处理连续型特征，因此需要先将连续型特征离散化。这种转换会破坏连续数据的顺序关系，因此可能影响模型的效果。

**优缺点**：

- **优点**：简单直观，易于实现。
- **缺点**：不能直接处理连续型特征；多叉分割方式导致树的结构较复杂，容易过拟合。

#### 2. C4.5

##### 信息增益率 (Information Gain Ratio)
信息增益率是信息增益和特征的**固有值**（Intrinsic Value）的比值，用来平衡特征取值数对信息增益的影响。

1. **信息增益** $IG(D, A)$ 按照 ID3 的方式计算。

2. **固有值** (Intrinsic Value)：特征 $A$ 的固有值 $IV(A)$ 定义为：

   $$
   IV(A) = -\sum_{j=1}^v \frac{|D_j|}{|D|} \log_2 \left( \frac{|D_j|}{|D|} \right)
   $$

3. **信息增益率** $GR(D, A)$ 定义为信息增益与固有值的比值：

   $$
   GR(D, A) = \frac{IG(D, A)}{IV(A)}
   $$


**C4.5** 是 ID3 的改进版，它引入了**信息增益率**的概念，以克服 ID3 在特征选择上的缺陷。其主要特征如下：

- **特征选择**：C4.5 使用**信息增益率**来选择分裂特征。信息增益率修正了信息增益的偏差（即倾向于选择取值较多的特征）。信息增益率更关注特征的相对信息增益，从而使模型更稳健。
  
- **分支策略**：C4.5 同样支持多分支切分，即如果特征有多个取值，会创建相应数量的分支。
  
- **连续型特征处理**：C4.5 能直接处理连续型特征。它通过在每个连续特征上选择一个最佳的切分点，将连续型特征转化为二元分类。例如，如果特征值大于某个阈值，则走左分支，否则走右分支。

**优缺点**：

- **优点**：可以处理连续型特征，模型更稳定，不易过拟合。
- **缺点**：计算量较大，特别是在计算信息增益率和选择最佳切分点时。

#### 3. CART（Classification and Regression Tree）

##### 分类任务：基尼指数 (Gini Index)
在分类任务中，CART 使用**基尼指数**来衡量数据集的纯度。基尼指数越小，数据集的纯度越高。

对于一个数据集 $D$，基尼指数 $G(D)$ 定义为：

$$
G(D) = 1 - \sum_{i=1}^k p_i^2
$$

其中，$p_i$ 表示类别 $i$ 的样本占比。

假设特征 $A$ 将数据集 $D$ 分割为 $D_1$ 和 $D_2$ 两个子集，则基于特征 $A$ 的基尼指数定义为：

$$
G(D, A) = \frac{|D_1|}{|D|} G(D_1) + \frac{|D_2|}{|D|} G(D_2)
$$

CART 算法会选择基尼指数最小的特征进行分割。

##### 回归任务：最小方差 (Minimum Variance)
在回归任务中，CART 选择分割特征的标准是**最小方差**。假设数据集 $D$ 中的均值为 $\mu$，则方差 $Var(D)$ 定义为：

$$
Var(D) = \sum_{i=1}^n (x_i - \mu)^2
$$

**CART** 是一种更通用的决策树算法，可以用于分类和回归。其主要特征如下：

- **特征选择**：在分类任务中，CART 使用**基尼指数**来衡量分裂节点的质量。基尼指数衡量样本的纯度，值越小表示数据越纯。在回归任务中，CART 则使用**最小方差**来选择分裂节点。
  
- **分支策略**：CART 使用**二元分割**，即每次将数据集分割为两部分。二元分割使得树的结构更简单，可以更高效地处理连续型特征。对于连续特征，CART 会寻找一个最佳的切分点，将数据分为大于或小于该值的两部分。
  
- **连续型特征处理**：CART 能直接处理连续型特征，通过设置阈值，将数据集二分。

**优缺点**：

- **优点**：支持连续型特征，结构更清晰，易于处理连续数据，且能够同时用于分类和回归。
- **缺点**：由于使用二元分割，树的深度可能较深，计算量较大，且在多分类问题中可能表现不如多分支的 C4.5。

#### 各算法的特征选择方式与分支策略

| 算法 | 特征选择依据    | 分支方式       | 是否支持连续特征 |
|------|-----------------|----------------|------------------|
| ID3  | 信息增益       | 多分支         | 否              |
| C4.5 | 信息增益率     | 多分支         | 是              |
| CART | 基尼指数（分类）/ 最小方差（回归） | 二元分支 | 是              |

#### 应用拓展

基于 CART 构建的树模型还发展出了一些集成算法：

- **随机森林（Random Forest）**：使用 CART 作为基学习器，通过集成大量决策树来提高模型的稳定性和泛化能力。
- **梯度提升决策树（GBDT）**：使用回归树作为基学习器，通过梯度提升框架来逐步减小模型误差，通常用于回归问题和分类问题。

#### 总结

- **ID3**：基于信息增益，使用多分支切分，适合处理离散型特征。
- **C4.5**：改进了 ID3，使用信息增益率，支持连续型特征，分类效果更好。
- **CART**：采用二元分支，支持连续型和离散型特征，既可以用于分类也可以用于回归，是随机森林和 GBDT 等集成方法的基础。

这些算法各有优劣，实际应用中可以根据数据的特点和任务需求来选择合适的树构建算法。

### 工作原理
1. 找到数据集切分的最佳位置
   -  对每个特征:
      - 对每个特征值: 
         -  将数据集切分成两份（小于该特征值的数据样本放在左子树，否则放在右子树）
         -  计算切分的误差
         -  如果当前误差小于当前最小误差，那么将当前切分设定为最佳切分并更新最小误差
   -  返回最佳切分的特征和阈值

2. 树构建算法
   -  找到最佳的待切分特征:
      -  如果该节点不能再分，将该节点存为叶节点
      -  执行二元切分
      -  在右子树调用 createTree() 方法
      -  在左子树调用 createTree() 方法

### 剪枝
#### 预剪枝
预剪枝是一种在树构建过程中及早停止生长的剪枝方法。其基本思想是，在决策树的构建过程中引入剪枝规则，以避免树的深度增长过大。预剪枝主要通过提前终止条件来控制树的生长，比如：
-  设定阈值：如果某个节点的划分后信息增益或熵的减少量小于设定的阈值，则停止对该节点的进一步分裂。
-  最小样本数限制：设定分裂后节点的最小样本数，如果分裂出的子节点中样本数低于此值，则停止继续分裂。

在函数 find_best_split() 中，通过设置参数（如误差减少的阈值和最小分割样本数）实现了预剪枝的机制。这种机制实际上通过阈值控制，减少决策树的增长，从而避免因过度分裂而导致的过拟合。

缺点：预剪枝的效果不总是理想，因为有时候通过设置阈值限制，可能会过早地停止树的增长，导致模型没有充分学习数据的分布特征，导致模型欠拟合。

#### 后剪枝
后剪枝是在决策树构建完成后进行剪枝的过程。后剪枝更常见，因为它允许决策树在初始构建时尽量拟合数据，然后再对复杂度进行控制。后剪枝的过程通常如下：
-  利用验证集进行剪枝：决策树构造完成后，利用验证集逐步评估每个分支节点。如果删除某一分支节点，并将其所有子节点合并为一个叶节点后模型的误差没有显著增加，则可以删除该分支。
-  熵的变化：判断拥有相同父节点的多个子节点，将其合并后，熵的增加量是否小于某一设定的阈值。如果确实小于，则合并这些子节点，这一过程称为“塌陷处理”。

在回归树中，后剪枝的一个常见方法是将多个子节点的值取平均值，作为合并后的叶节点值。通过这种方式，可以保留较重要的分支，同时合并那些细微的、不显著的分支，从而降低树的复杂度。

优点：后剪枝由于在整个树构建完成后再进行剪枝，因此往往可以更好地处理复杂数据集，防止模型的过拟合。

In [11]:
import numpy as np

In [12]:
def load_data_set(fileName):
    dataMat = []
    with open(fileName) as f:
        for line in f:
            values = list(map(float, line.strip().split('\t')))
            dataMat.append(values)
    return np.array(dataMat)

def binary_split_data_set(dataMat, featureIdx, value):
    """
    按照指定特征列和特征值，对数据集进行二元切分
    参数:
        data_mat: 原始数据集
        feature_idx: 要切分的特征列的索引
        value: 特征列中要比较的值
    返回:
        left_set: 小于等于 value 的子集
        right_set: 大于 value 的子集
    """
    leftSet = dataMat[dataMat[:, featureIdx] <= value]
    rightSet = dataMat[dataMat[:, featureIdx] > value]
    return leftSet, rightSet

def leaf_mean(dataMat):
    return np.mean(dataMat[:, -1])

def error_variance(dataMat):
    """计算数据集的误差方差，误差等于方差乘以样本数"""
    return np.var(dataMat[:, -1]) * dataMat.shape[0]

def find_best_split(dataMat, leafFunc=leaf_mean, errorFunc=error_variance, ops=(1, 4)):
    """
    寻找数据集的最佳切分点并返回
    参数:
        data_set: 输入数据集，假设为 NumPy 数组，最后一列为目标值
        leaf_func: 叶节点计算函数，用于计算当达到停止条件时叶节点的值
        error_func: 误差计算函数，用于计算数据集的误差，通常使用方差乘以样本数
        ops: 控制划分停止的参数元组 (最小误差下降值, 最小分割样本数)
            - tol_error_reduction (ops[0]): 切分后误差减少的最小值，小于该值则停止切分
            - min_sample_size (ops[1]): 切分后的数据集最小样本数，小于该值则停止切分
    返回:
        best_feature_idx: 最佳特征列的索引，如果不满足切分条件，则返回 None
        best_split_val: 最佳切分值，如果不满足切分条件，则返回叶节点的值
    """
    # 解包 ops，设置误差减少的最小值和最小分割样本数
    totalErrorReduction, minSaple = ops

    # 如果所有值相等，则返回叶节点
    if len(set(dataMat[:, -1])) == 1:
        return None, leafFunc(dataMat)
    
    currentError = errorFunc(dataMat)
    bestError = np.inf
    bestFeatureIdx = 0
    bestSplitVal = 0
    m, n = dataMat.shape

    for featureIdx in range(n - 1):
        # 对当前特征列的所有唯一值进行遍历，用于作为候选切分值
        for splitVal in set(dataMat[:, featureIdx]):
            leftSet, rightSet = binary_split_data_set(dataMat, featureIdx, splitVal)
            # 如果切分后的数据集样本数小于最小分割样本数，则跳过
            if leftSet.shape[0] < minSaple or rightSet.shape[0] < minSaple:
                continue

            # 计算切分后的误差
            newError = errorFunc(leftSet) + errorFunc(rightSet)
            if newError < bestError:
                bestError = newError
                bestFeatureIdx = featureIdx
                bestSplitVal = splitVal

    # 如果误差减少不大，则返回叶节点
    if currentError - bestError < totalErrorReduction:
        return None, leafFunc(dataMat)
    
    # 再次验证切分后的数据集样本数是否小于最小分割样本数
    leftSet, rightSet = binary_split_data_set(dataMat, bestFeatureIdx, bestSplitVal)
    if leftSet.shape[0] < minSaple or rightSet.shape[0] < minSaple:
        return None, leafFunc(dataMat)
    
    return bestFeatureIdx, bestSplitVal

def build_tree(dataMat, leafFunc=leaf_mean, errorFunc=error_variance, ops=(1, 4)):
    """
    递归构建回归树
    参数:
        data_set: 输入数据集
        leaf_func: 叶节点计算函数
        error_func: 误差计算函数
        ops: 控制划分停止的参数 (最小误差下降值, 最小分割样本数)
    返回:
        tree: 回归树的字典表示
    """

    featureIdx, splitVal = find_best_split(dataMat, leafFunc, errorFunc, ops)
    if featureIdx is None:
        return splitVal
    
    tree = {'feature_idx': featureIdx, 'split_val': splitVal}
    leftSet, rightSet = binary_split_data_set(dataMat, featureIdx, splitVal)
    tree['left'] = build_tree(leftSet, leafFunc, errorFunc, ops)
    tree['right'] = build_tree(rightSet, leafFunc, errorFunc, ops)
    return tree

def is_tree(obj):
    """判断输入对象是否是树的字典结构"""
    return isinstance(obj, dict)

def collapse_tree(tree):
    """
    从上往下遍历树直到叶节点，合并叶节点为均值
    参数:
        tree: 输入的树
    返回:
        均值: 合并后的均值
    """
    if is_tree(tree['left']):
        tree['left'] = collapse_tree(tree['left'])
    if is_tree(tree['right']):
        tree['right'] = collapse_tree(tree['right'])
    return (tree['left'] + tree['right']) / 2.0

def prune_tree(tree, testData):
    """
    使用测试数据对树进行剪枝
    参数:
        tree: 需要剪枝的树
        test_data: 剪枝所需的测试数据
    返回:
        tree: 剪枝后的树
    """

    # 如果测试数据为空，则直接对树进行塌陷处理
    if len(testData) == 0:
        return collapse_tree(tree)
    
    # 如果左右分支有任意一个是树，按当前切分特征值划分测试数据
    if is_tree(tree['left']) or is_tree(tree['right']):
        leftSet, rightSet = binary_split_data_set(testData, tree['feature_idx'], tree['split_val'])
    
    if is_tree(tree['left']):
        tree['left'] = prune_tree(tree['left'], leftSet)
    
    if is_tree(tree['right']):
        tree['right'] = prune_tree(tree['right'], rightSet)

    # 如果左右分支都是叶节点，计算合并前后的误差
    if not is_tree(tree['left']) and not is_tree(tree['right']):
        leftSet, rightSet = binary_split_data_set(testData, tree['feature_idx'], tree['split_val'])

        # 合并前的误差
        noMergeError = np.sum(np.power(leftSet[:, -1] - tree['left'], 2)) + np.sum(np.power(rightSet[:, -1] - tree['right'], 2))

        # 合并后的均值
        mergeValue = (tree['left'] + tree['right']) / 2.0
        mergeError = np.sum(np.power(testData[:, -1] - mergeValue, 2))

        # 如果合并后的误差小于不合并的误差，则合并
        if mergeError < noMergeError:
            return mergeValue
    
    return tree

def predict(tree, data, evalFunc=lambda model, data: float(model)):
    """
    使用树对数据进行预测
    参数:
        tree: 回归树或模型树
        data: 输入数据
        eval_func: 叶节点预测的计算方式，默认为转换成浮点数输出
    返回:
        预测值
    """
    if not is_tree(tree):
        return evalFunc(tree, data)
    
    if data[tree['feature_idx']] <= tree['split_val']:
        return predict(tree['left'], data, evalFunc)
    else:
        return predict(tree['right'], data, evalFunc)
    
def batch_predict(tree, dataMat, evalFunc=lambda model, data: float(model)):
    """
    批量预测
    参数:
        tree: 回归树或模型树
        data_mat: 输入数据集
        eval_func: 叶节点预测的计算方式，默认为转换成浮点数输出
    返回:
        预测值列表
    """
    return np.array([predict(tree, data, evalFunc) for data in dataMat])

In [13]:
# if __name__ == '__main__':
#     dataMat = load_data_set('data1.txt')
#     tree = build_tree(dataMat)
#     print(tree)
#     print('---' * 20)
#     print('prune tree')
#     print(prune_tree(tree, dataMat))

# if __name__ == '__main__':
#     dataMat = load_data_set('data2.txt')
#     tree = build_tree(dataMat, ops=(0, 1))
#     print(tree)
#     print('---' * 20)
#     print('prune tree')
#     print(prune_tree(tree, dataMat))

## 模型树
用树来对数据建模，除了把叶节点简单地设定为常数值之外，还有一种方法是把叶节点设定为分段线性函数，这里所谓的 分段线性（piecewise linear） 是指模型由多个线性片段组成。

模型树的可解释性是它优于回归树的特点之一。另外，模型树也具有更高的预测准确度。

将之前的回归树的代码稍作修改，就可以在叶节点生成线性模型而不是常数值。

前面用于回归树的误差计算方法这里不能再用。稍加变化，对于给定的数据集，应该先用模型来对它进行拟合，然后计算真实的目标值与模型预测值间的差值。最后将这些差值的平方求和就得到了所需的误差。

In [14]:
def linear_solve(dataSet):
    m, n = dataSet.shape
    X = np.ones((m, n))  # 创建一个全为1的矩阵，用于常数项
    Y = dataSet[:, -1].reshape(-1, 1)  # 目标变量Y为数据集的最后一列，并转换为列向量
    X[:, 1:] = dataSet[:, :-1]  # 自变量矩阵X除第一列外，其余列为数据集的前n-1列

    xTx = X.T @ X  # 自变量矩阵X的转置与X相乘
    if np.linalg.det(xTx) == 0.0:  # 判断矩阵是否可逆
        raise ValueError('矩阵为奇异矩阵，不可逆。请调整参数或检查数据。')
    
    ws = np.linalg.inv(xTx) @ (X.T @ Y)  # 计算最小二乘法的回归系数ws
    return ws, X, Y

def model_leaf(dataSet):
    ws, _, _ = linear_solve(dataSet)
    return ws

def modelErr(dataSet):
    """
    计算数据集上的总误差。
    
    参数:
        dataSet -- 输入数据集，包含自变量和目标变量
    
    返回:
        平方误差的总和，衡量模型的误差
    """
    ws, X, Y = linear_solve(dataSet)
    yHat = X @ ws  # 使用@符号表示矩阵乘法
    return np.sum(np.square(Y - yHat))  # 返回误差平方和

if __name__ == '__main__':
    dataMat = load_data_set('data4.txt')
    tree = build_tree(dataMat, leafFunc=model_leaf, errorFunc=modelErr, ops=(1, 10))
    